In [8]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import numpy as np
import random #as rd
import copy
import requests
import io
from PIL import Image,ImageTk
from ttkbootstrap import Style

In [9]:
df = pd.read_excel('fooddata.xlsx')
class GA:
    def __init__(self,df,pop_size=30,chromosome_length=6,max_generations=1000,crossover_rate=0.8,mutate_rate=0.05,best_cal=300):
        self.foods=df['食物名稱']
        self.cal=df['熱量']
        self.pop_size=pop_size
        self.chromosome_length=chromosome_length
        self.max_generations=max_generations
        self.crossover_rate=crossover_rate
        self.mutate_rate=mutate_rate
        self.best_cal=best_cal
    def generate_initial_population(self):
        # 初始化初始族群
        population = []
        for _ in range(self.pop_size):
            chromosome = [random.randint(0, 1) for _ in range(self.chromosome_length)]
            population.append(chromosome)
        return population
    
    def fitfunc(self,population):
        chromosome_d=np.zeros(len(population))
        fitness=np.zeros(len(population))
        #將二進位編碼轉為十進位
        for i in range(len(population)):
            chromosome_str="".join([str(_) for _ in population[i]])
            chromosome_d[i]=int(chromosome_str,2)
        #計算適應值
        for i in range(len(population)):
            fitness[i]=(self.best_cal-abs(self.best_cal-self.cal[chromosome_d[i]]))/\
            self.best_cal
        return fitness
    
    def selection(self,population):
        # 輪盤式選擇
        fitness=self.fitfunc(population)
        total_fitness=sum(fitness)
        probabilities=fitness/total_fitness
        new_population=[]
        for i in range(15):
            selected_chromosomes = random.choices(population, probabilities, k=2)
            a,b=self.crossover(selected_chromosomes)
            new_population.append(a)
            new_population.append(b)
        new_population2=self.mutation(new_population)
        return new_population2
    
    def crossover(self,selected_chromosomes):
        # 雙點交配
        if random.random()<self.crossover_rate:
            #隨機選擇不重複兩點
            crossover_point1 = random.randint(0, self.chromosome_length - 1)
            crossover_point2 = random.randint(crossover_point1 + 1, self.chromosome_length)
            #將選中的基因段互換
            new_chromosome1 = selected_chromosomes[0][:crossover_point1] + \
            selected_chromosomes[1][crossover_point1:crossover_point2] + \
            selected_chromosomes[0][crossover_point2:]
            new_chromosome2 = selected_chromosomes[1][:crossover_point1] + \
            selected_chromosomes[0][crossover_point1:crossover_point2] + \
            selected_chromosomes[1][crossover_point2:]
        else:
            new_chromosome1=selected_chromosomes[0]
            new_chromosome2=selected_chromosomes[1]
        return new_chromosome1, new_chromosome2
    
    def mutation(self,population):
        # 雙點突變
        new_population=population[:]
        for i in range(len(population)):
            mutated_chromosome = population[i]
            if random.random() < self.mutate_rate:
                mutation_point=random.sample(range(0,self.chromosome_length),2)
                mutated_chromosome[mutation_point[0]]= 1 | mutated_chromosome[mutation_point[0]]
                mutated_chromosome[mutation_point[1]]= 1 | mutated_chromosome[mutation_point[1]]
                new_population[i]=mutated_chromosome
            
        return new_population
    
    def best_solution(self):
        population=self.generate_initial_population()
        for _ in range(self.max_generations):
            population=self.selection(population)
            fitness=self.fitfunc(population)
            for i in range(1, len(population)):
                if fitness[i]>fitness[i-1]:
                    best=population[i]
                    best_fitness=fitness[i]
                elif i==1:
                    best=population[0]
                    best_fitness=fitness[0]
            if best_fitness>0.95:
                break
        best_str="".join([str(_) for _ in best])
        best_chromosome=int(best_str,2)
        
        return self.foods[best_chromosome],self.cal[best_chromosome]


In [10]:
class UI:
    def __init__(self,window):
        self.window=window
        self.window.title("711減肥食物推薦系統")
        self.window.geometry("1000x800")
        self.window.configure(bg="white") #背景顏色
        self.first_page()
    def first_page(self):
        self.clear_frame()
        # 建立主視窗
        
        #window = tk.Tk()
        
        # 設定視窗大小
        
        # 左側圖像
        url="https://d1csarkz8obe9u.cloudfront.net/posterpreviews/food-icon-2-design-template-b7f020f0195178f74a0a241024343dd3_screen.jpg?ts=1627156857"
        image_bytes=requests.get(url).content
        data_stream=io.BytesIO(image_bytes)
        pil_image=Image.open(data_stream)
        global tk_image
        tk_image=ImageTk.PhotoImage(pil_image)
        image_label=tk.Label(window,image=tk_image,bg="white")
        image_label.pack(side=tk.LEFT)

        # 右側表單
        self.form_frame = tk.Frame(self.window)
        self.form_frame.pack(side=tk.LEFT, padx=10, pady=10)
        self.form_frame.configure(bg="white") #背景顏色
        
        global gender_var,weight_entry,age_entry,activity_level_var
        global height_entry
        # 性別
        gender_label = tk.Label(self.form_frame, text="性別:",bg="white")
        gender_label.grid(row=0, column=0, sticky=tk.W)
        gender_var = tk.StringVar()
        gender_combobox = ttk.Combobox(self.form_frame, textvariable=gender_var, state="readonly")
        gender_combobox["values"] = ("男", "女")
        gender_combobox.grid(row=0, column=1, padx=10, pady=5)

        # 體重
        weight_label = tk.Label(self.form_frame, text="體重 (公斤):",bg="white")
        weight_label.grid(row=3, column=0, sticky=tk.W)
        weight_entry = tk.Entry(self.form_frame)
        weight_entry.grid(row=3, column=1, padx=10, pady=5)

        # 年齡
        age_label = tk.Label(self.form_frame, text="年齡:",bg="white")
        age_label.grid(row=1, column=0, sticky=tk.W)
        age_entry = tk.Entry(self.form_frame)
        age_entry.grid(row=1, column=1, padx=10, pady=5)

        # 身高
        height_label = tk.Label(self.form_frame, text="身高 (公分):",bg="white")
        height_label.grid(row=2, column=0, sticky=tk.W)
        height_entry = tk.Entry(self.form_frame)
        height_entry.grid(row=2, column=1, padx=10, pady=5)

        # 運動強度
        activity_label = tk.Label(self.form_frame, text="運動強度:",bg="white")
        activity_label.grid(row=4, column=0, sticky=tk.W)
        activity_level_var = tk.StringVar()
        activity_level_combobox = ttk.Combobox(self.form_frame, textvariable=activity_level_var, state="readonly")
        activity_level_combobox["values"] = ("沒運動", "輕度運動", "中度運動", "高度運動")
        activity_level_combobox.grid(row=4, column=1, padx=10, pady=5)

        # 計算按鈕
        calculate_button = tk.Button(self.form_frame, text="計算", command=self.calculate_bmr ,width=10)
        calculate_button.grid(row=5, columnspan=2, pady=10)
    def calculate_bmr(self):
        weight = float(weight_entry.get())
        height = float(height_entry.get())
        age = int(age_entry.get())
        gender = gender_var.get()
        if gender=="男":
            gender_int=1
        else:
            gender_int=0
        activity_level = activity_level_var.get()
        if activity_level=="沒運動":
            activity=1.2
        elif activity_level=="輕度運動":
            activity=1.375
        elif activity_level=="中度運動":
            activity=1.55
        else:
            activity=1.72
        BMR=9.99*weight+6.25*height-4.92*age+166*gender_int-161
        TDEE=BMR*activity
        self.second_page(BMR,TDEE)
    def second_page(self,BMR,TDEE):
        self.clear_frame()
       
        self.form_frame = tk.Frame(self.window)
        self.form_frame.pack(side=tk.LEFT, padx=10, pady=10)
        self.form_frame.configure(bg="white")
        
        
         # 左側圖像
        url4="https://img.freepik.com/free-vector/lifting-weights-design_1133-261.jpg?w=826&t=st=1685534074~exp=1685534674~hmac=158af98598913d145b8abd2fc946d7b7398e98fe35e75008fcb422ecab74c362"
        image_bytes4=requests.get(url4).content
        data_stream4=io.BytesIO(image_bytes4)
        pil_image4=Image.open(data_stream4)
        pil_image4=pil_image4.resize((150,150))
        global tk_image4
        tk_image4=ImageTk.PhotoImage(pil_image4)
        image_label4=tk.Label(self.form_frame,image=tk_image4,bg="white")
        image_label4.grid(row=0, column=0,rowspan=2)
        
        url1="https://cdn-icons-png.flaticon.com/512/1117/1117390.png?w=826&t=st=1685530695~exp=1685531295~hmac=bd068e6b1ab0848c5794c5ea0030519d6768179ebed673487768e2eed1d64f48"
        image_bytes1=requests.get(url1).content
        data_stream1=io.BytesIO(image_bytes1)
        pil_image1=Image.open(data_stream1)
        pil_image1=pil_image1.resize((150,150))
        global tk_image1
        tk_image1=ImageTk.PhotoImage(pil_image1)
        image_label1=tk.Label(self.form_frame,image=tk_image1,bg="white")
        image_label1.grid(row=2, column=0,rowspan=2)
        
        url2="https://cdn-icons-png.flaticon.com/512/267/267916.png?w=826&t=st=1685530985~exp=1685531585~hmac=49438ff873efea651c43be90d58fae201e72eeb9e6abfe7e3bf13b6dba1eb1e0"
        image_bytes2=requests.get(url2).content
        data_stream2=io.BytesIO(image_bytes2)
        pil_image2=Image.open(data_stream2)
        pil_image2=pil_image2.resize((150,150))
        global tk_image2
        tk_image2=ImageTk.PhotoImage(pil_image2)
        image_label2=tk.Label(self.form_frame,image=tk_image2,bg="white")
        image_label2.grid(row=4, column=0,rowspan=2)
        
        url3="https://cdn-icons-png.flaticon.com/512/661/661685.png?w=826&t=st=1685530760~exp=1685531360~hmac=cc237eeceae8774794d0e1f872af13678c5845564ebd54cb90fdba958f7bc025"
        image_bytes3=requests.get(url3).content
        data_stream3=io.BytesIO(image_bytes3)
        pil_image3=Image.open(data_stream3)
        pil_image3=pil_image3.resize((150,150))
        global tk_image3
        tk_image3=ImageTk.PhotoImage(pil_image3)
        image_label3=tk.Label(self.form_frame,image=tk_image3,bg="white")
        image_label3.grid(row=6, column=0,rowspan=2)
        
        return_msg = tk.StringVar()
        return_msg.set("　　 BMR = " + str(BMR) + "\n" + "　　　 TDEE = " + str(TDEE))
        result_label = tk.Label(self.form_frame, textvariable=return_msg ,bg="white",font=("思源黑體",20))
        result_label.grid(row=0, column=2, columnspan=4,rowspan=2,sticky=tk.W)

        '''if (TDEE-BMR)<300:
            best=TDEE-100
        elif (TDEE-BMR)<500:
            best=TDEE-200
        else:
            best=TDEE-300'''
        best=(BMR+TDEE)/2
        breakfast_cal=best*2/9
        lunch_cal=best*4/9
        dinner_cal=best*3/9
        print(breakfast_cal,lunch_cal,dinner_cal)

        a=GA(df,pop_size=30,chromosome_length=6,max_generations=1000,crossover_rate=0.8,mutate_rate=0.05,best_cal=breakfast_cal)
        breakfast_food,breakfast_food_cal=a.best_solution()
        print(breakfast_food,breakfast_food_cal)
        breakfast_msg = tk.StringVar()
        breakfast_msg1=tk.StringVar()
        breakfast_msg.set("　　　早餐推薦食物: " + breakfast_food )
        breakfast_msg1.set("　　　熱量: " + str(breakfast_food_cal) +"kcal")
        breakfast_label = tk.Label(self.form_frame, textvariable=breakfast_msg ,bg="white",font=("思源黑體",20))
        breakfast_label.grid(row=2, column=2, columnspan=4, sticky=tk.W)
        breakfast_label1 = tk.Label(self.form_frame, textvariable=breakfast_msg1 ,bg="white",font=("思源黑體",20))
        breakfast_label1.grid(row=3, column=2, columnspan=4, sticky=tk.W)

        lunch_msg = tk.StringVar()
        lunch_msg1 = tk.StringVar()
        if lunch_cal>800:
            lunch_cal1=lunch_cal*2/5
            lunch_cal2=lunch_cal*3/5
            print(lunch_cal1,lunch_cal2)
            b1=GA(df,pop_size=30,chromosome_length=6,max_generations=1000,crossover_rate=0.8,mutate_rate=0.05,best_cal=lunch_cal1)
            lunch_food1,lunch_food_cal1=b1.best_solution()
            b2=GA(df,pop_size=30,chromosome_length=6,max_generations=1000,crossover_rate=0.8,mutate_rate=0.05,best_cal=lunch_cal2)
            lunch_food2,lunch_food_cal2=b2.best_solution()
            print(lunch_food1,lunch_food2,lunch_food_cal1,lunch_food_cal2)
            lunch_food_cal=lunch_food_cal1+lunch_food_cal2
            lunch_msg.set("　　　午餐推薦食物: " + lunch_food1 + "+" +lunch_food2)
            lunch_msg1.set("　　　熱量: " + str(lunch_food_cal) +"kcal")

        else:
            b=GA(df,pop_size=30,chromosome_length=6,max_generations=1000,crossover_rate=0.8,mutate_rate=0.05,best_cal=lunch_cal)
            lunch_food,lunch_food_cal=b.best_solution()
            print(lunch_food,lunch_food_cal)
            lunch_msg.set("　　　午餐推薦食物: " + lunch_food)
            lunch_msg1.set("　　　熱量: " + str(lunch_food_cal) +"kcal")
        lunch_label = tk.Label(self.form_frame, textvariable=lunch_msg ,bg="white",font=("思源黑體",20))
        lunch_label.grid(row=4, column=2, columnspan=4, sticky=tk.W)
        lunch_label1 = tk.Label(self.form_frame, textvariable=lunch_msg1 ,bg="white",font=("思源黑體",20))
        lunch_label1.grid(row=5, column=2, columnspan=4, sticky=tk.W)

        dinner_msg = tk.StringVar()
        dinner_msg1 = tk.StringVar()
        if dinner_cal>800:
            dinner_cal1=dinner_cal*2/5
            dinner_cal2=dinner_cal*3/5
            c1=GA(df,pop_size=30,chromosome_length=6,max_generations=1000,crossover_rate=0.8,mutate_rate=0.05,best_cal=dinner_cal1)
            dinner_food1,dinner_food_cal1=c1.best_solution()
            c2=GA(df,pop_size=30,chromosome_length=6,max_generations=1000,crossover_rate=0.8,mutate_rate=0.05,best_cal=dinner_cal2)
            dinner_food2,dinner_food_cal2=c2.best_solution()
            print(dinner_food1,dinner_food2,dinner_food_cal1,dinner_food_cal2)
            dinner_food_cal=dinner_food_cal1+dinner_food_cal2
            dinner_msg.set("　　　晚餐推薦食物: " + dinner_food1 + "+" +dinner_food2)
            dinner_msg1.set("　　　熱量: " + str(dinner_food_cal) +"kcal")
        else:  
            c=GA(df,pop_size=30,chromosome_length=6,max_generations=1000,crossover_rate=0.8,mutate_rate=0.05,best_cal=dinner_cal)
            dinner_food,dinner_food_cal=c.best_solution()
            print(dinner_food,dinner_food_cal)
            dinner_msg.set("　　　晚餐推薦食物: " + dinner_food )
            dinner_msg1.set("　　　熱量: " + str(dinner_food_cal) +"kcal")
        dinner_label = tk.Label(self.form_frame, textvariable=dinner_msg ,bg="white",font=("思源黑體",20))
        dinner_label.grid(row=6, column=2, columnspan=4, sticky=tk.W)
        dinner_label1 = tk.Label(self.form_frame, textvariable=dinner_msg1 ,bg="white",font=("思源黑體",20))
        dinner_label1.grid(row=7, column=2, columnspan=4, sticky=tk.W)

        text_msg = tk.StringVar()
        if (breakfast_food_cal+lunch_food_cal+dinner_food_cal)<BMR:
            text_msg.set("\n" + "　　　對不起(╥﹏╥)，推薦食物的熱量小於BMR，請再多補充一些熱量呦！")
        elif (breakfast_food_cal+lunch_food_cal+dinner_food_cal)>TDEE:
            text_msg.set("\n" + "　　　對不起(╥﹏╥)，推薦食物的熱量大於TDEE，請少攝取一些熱量才能順利減肥呦！")
        else:
            text_msg.set("\n" + "　　　減肥的路上有你有我，加油！٩(◦`꒳´◦)۶")
        text_label=tk.Label(self.form_frame, textvariable=text_msg ,bg="white",font=("思源黑體",20))
        text_label.grid(row=10, column=2, columnspan=4,rowspan=2, sticky=tk.W)
        back_button = tk.Button(self.form_frame, text="RESTART", command=self.back ,width=25)
        back_button.grid(row=12, column=2 ,columnspan=4,pady=20)
    def clear_frame(self):
        for widget in self.window.winfo_children():
            widget.destroy()
    def back(self):
        self.first_page()

In [11]:
#style=Style(theme="minty")
#ttt=style.master
window=tk.Tk()
ui=UI(window)
window.mainloop()